In [1]:
import os
import sys

module_path = os.path.abspath("..")
if module_path not in sys.path:
    sys.path.append(module_path)

import tensorflow as tf
from tensorflow.keras import backend as K

import numpy as np

# MultipleNgramConvs like in PACRR

https://arxiv.org/abs/1704.03940

In [2]:
from interactions import SemanticInteractions
from local_relevance import MultipleNgramConvs

In [3]:
# test
K.clear_session()

emb_matrix = np.random.random((10000,68))

input_query = tf.keras.layers.Input((8,), dtype="int32")
input_sentence = tf.keras.layers.Input((4,), dtype="int32")

semantic_interaction = SemanticInteractions(emb_matrix)
extendedpacrr = MultipleNgramConvs(max_ngram=3, k_max=2)

_out = semantic_interaction([input_query, input_sentence])
_out = extendedpacrr(_out)

model = tf.keras.models.Model(inputs=[input_query, input_sentence], outputs=_out)
model.summary()

[EMBEDDING MATRIX SHAPE] (10000, 68)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
semantic_interactions (Semantic (None, 8, 4, 3)      136         input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
multiple_ngram_convs (MultipleN (None, 8, 18)        2352

In [14]:
# test with some values

query = np.random.randint(1,4,(1, 8))
document = np.random.randint(1,4,(1, 4))

y = model.predict([query, document])
print("print colums from the matrix")
print(y[0,:,0])
print(y[0,:,1])
print(y[0,:,2])
print(y[0,:,-2])
print(y[0,:,-1])

[21.912245 17.800749 15.643004 21.912245 17.800749 15.643004 17.800749
 17.800749]
[21.912245 17.800749 15.643004 21.912245 17.800749 15.643004 17.800749
 17.800749]
[8.236326  6.608107  5.8339515 8.236326  6.608107  5.8339515 6.608107
 6.608107 ]
[5.537063  6.4712768 7.1589804 6.694434  6.4712768 6.1526017 6.3497453
 5.6416974]
[5.1988726 6.354459  6.6964936 6.3935385 6.354459  6.068408  5.989276
 5.306098 ]


In [15]:
query[:,4:] = np.zeros(4,)
document[:,2:] = np.zeros(2,)

y = model.predict([query, document])
print("print colums from the matrix")
print(y[0,:,0])
print(y[0,:,1])
print(y[0,:,2])
print(y[0,:,-2])
print(y[0,:,-1])

print colums from the matrix
[21.912245 14.866847 15.643004 21.912245  0.        0.        0.
  0.      ]
[21.912245 14.866847 15.643004 21.912245  0.        0.        0.
  0.      ]
[8.236326  5.5318522 5.8339515 8.236326  0.        0.        0.
 0.       ]
[5.4462748 5.750807  6.2426004 5.605811  0.        0.        0.
 0.       ]
[4.466276  5.4981523 5.200652  3.8984463 0.        0.        0.
 0.       ]


# MaskedSoftmax

In [2]:
from local_relevance import MaskedSoftmax

In [8]:
K.clear_session()


query = np.random.randint(1,8,(1, 10)).astype(np.float32)
query[:,5:] = np.zeros(5,)
idf_softmax = MaskedSoftmax()

print(query)
print(idf_softmax(query))

norm = query/np.sum(query)
print("norm",norm)
print(idf_softmax(norm))


[[3. 4. 3. 3. 7. 0. 0. 0. 0. 0.]]
call tf.Tensor(
[[ 3.e+00  4.e+00  3.e+00  3.e+00  7.e+00 -1.e+09 -1.e+09 -1.e+09 -1.e+09
  -1.e+09]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[0.01657923 0.04506702 0.01657923 0.01657923 0.90519524 0.
  0.         0.         0.         0.        ]], shape=(1, 10), dtype=float32)
norm [[0.15 0.2  0.15 0.15 0.35 0.   0.   0.   0.   0.  ]]
call tf.Tensor(
[[ 1.5e-01  2.0e-01  1.5e-01  1.5e-01  3.5e-01 -1.0e+09 -1.0e+09 -1.0e+09
  -1.0e+09 -1.0e+09]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[0.1896571  0.19938102 0.1896571  0.1896571  0.2316477  0.
  0.         0.         0.         0.        ]], shape=(1, 10), dtype=float32)


In [11]:
## model simple outputs

K.clear_session()

input_query = tf.keras.layers.Input((10,), dtype="float32")

idf_softmax = MaskedSoftmax()

_out = idf_softmax(input_query)

model = tf.keras.models.Model(inputs=input_query, outputs=_out)
model.summary()

call Tensor("masked_softmax/sub:0", shape=(None, 10), dtype=float32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
masked_softmax (MaskedSoftma (None, 10)                0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [12]:

print(query)
print(model.predict(query))

norm = query/np.sum(query)
print("norm",norm)
print(model.predict(norm))

[[4 6 2 5 5 0 0 0 0 0]]
[[0.07162834 0.5292658  0.00969384 0.19470602 0.19470602 0.
  0.         0.         0.         0.        ]]
norm [[0.18181818 0.27272727 0.09090909 0.22727273 0.22727273 0.
  0.         0.         0.         0.        ]]
[[0.19602934 0.21468534 0.17899452 0.20514536 0.20514536 0.
  0.         0.         0.         0.        ]]
